# Solving a complex task with a multi-agent hierarchy


In [ ]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 15.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [ ]:
!pip install smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.

In [ ]:
import base64
import os

os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""

LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY")

if not LANGFUSE_PUBLIC_KEY or not LANGFUSE_SECRET_KEY:
    raise ValueError("Langfuse public or secret keys are missing!")

LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

#os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

print("Langfuse environment variables set successfully!")

Langfuse environment variables set successfully!


In [ ]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [ ]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool


model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

We can start with creating a baseline, simple agent to give us a simple report.

In [ ]:
task = """Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here (we're in Kolkata, 22.5744° N, 88.3629° E), and return them to me as a pandas dataframe.
Also give me some cricket stadiums if there are any nearby to the filming locations"""

In [ ]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [ ]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to     │
│ here (we're in Kolkata, 22.5744° N, 88.3629° E), and return them to me as a pandas dataframe.                   │
│ Also give me some cricket stadiums if there are any nearby to the filming locations                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:393: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:402: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(
ERROR:opentelemetry.exporter

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  hp_locations_text = web_search(query="all harry potter filming locations")                                       
  print(hp_locations_text)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Manually define coordinates for Harry Potter filming locations                                                 
  hp_coordinates = {                                                                                               
      "King's Cross Station, London": (51.5307, -0.1257),                                                          
      "Leadenhall Market, London": (51.5146, -0.0847),                                                             
      "Millennium Bridge, London": (51.5063, -0.0901),                                                             
      "Warner Bros. Studios, Leavesden": (51.7342, -0.2898),                                                       
      "Hampton Court Palace, London": (51.4785, -0.3298),                                                          
      "Alnwick Castle, Northumberland": (55.3705, -1.5938),                                                        
      "Durham Cathedral, Durham": (54.7768, -1.5729),                                                              
      "Gloucester Cathedral, Gloucestershire": (51.7414, -2.2413),                                                 
      "Christ Church, Oxford": (51.7575, -1.2447),                                                                 
      "Lacock Abbey, Wiltshire": (51.1059, -1.7761),                                                               
      "Edinburgh Castle, Edinburgh, Scotland": (55.9534, -3.1885),                                                 
      "Bodleian Library, Oxford": (51.7575, -1.2447)                                                               
  }                                                                                                                
                                                                                                                   
  # Extract travel time to Kolkata                                                                                 
  kolkata_coords = (22.5744, 88.3629)                                                                              
  travel_times = {}                                                                                                
                                                                                                                   
  # Function to calculate travel time using great-circle distance                                                  
  def calculate_travel_time(loc_coords):                                                                           
      if loc_coords:                                                                                               
          return calculate_cargo_travel_time(origin_coords=kolkata_coords, destination_coords=loc_coords)          
      return None                                                                                                  
                                                                                                                   
  # Calculate travel time for each location                                                                        
  for location, coords in hp_coordinates.items():                                                                  
      travel_times[location] = calculate_travel_time(coords)                                                       
                                                                                                                   
  # Extract nearby cricket stadiums                                                                                
  cricket_stadiums = {}                                  

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to 

Execution logs:
                                 Location         Coordinates  \
0            King's Cross Station, London  (51.5307, -0.1257)   
1               Leadenhall Market, London  (51.5146, -0.0847)   
2               Millennium Bridge, London  (51.5063, -0.0901)   
3         Warner Bros. Studios, Leavesden  (51.7342, -0.2898)   
4            Hampton Court Palace, London  (51.4785, -0.3298)   
5          Alnwick Castle, Northumberland  (55.3705, -1.5938)   
6                Durham Cathedral, Durham  (54.7768, -1.5729)   
7   Gloucester Cathedral, Gloucestershire  (51.7414, -2.2413)   
8                   Christ Church, Oxford  (51.7575, -1.2447)   
9                 Lacock Abbey, Wiltshire  (51.1059, -1.7761)   
10  Edinburgh Castle, Edinburgh, Scotland  (55.9534, -3.1885)   
11               Bodleian Library, Oxford  (51.7575, -1.2447)   

    Travel Time to Kolkata (hours) Nearby Cricket Stadiums  
0                            12.68                   ## Se  
1                            12.67                   ## Se  
2                            12.67                   ## Se  
3                            12.68                   ## Se  
4                            12.70                   ## Se  
5                            12.67                   ## Se  
6                            12.69                   ## Se  
7                            12.88                   ## Se  
8                            12.78                   ## Se  
9                            12.86                   ## Se  
10                           12.79                   ## Se  
11                           12.78                   ## Se  

Out - Final answer:                                  Location         Coordinates  \
0            King's Cross Station, London  (51.5307, -0.1257)   
1               Leadenhall Market, London  (51.5146, -0.0847)   
2               Millennium Bridge, London  (51.5063, -0.0901)   
3         Warner Bros. Studios, Leavesden  (51.7342, -0.2898)   
4            Hampton Court Palace, London  (51.4785, -0.3298)   
5          Alnwick Castle, Northumberland  (55.3705, -1.5938)   
6                Durham Cathedral, Durham  (54.7768, -1.5729)   
7   Gloucester Cathedral, Gloucestershire  (51.7414, -2.2413)   
8                   Christ Church, Oxford  (51.7575, -1.2447)   
9                 Lacock Abbey, Wiltshire  (51.1059, -1.7761)   
10  Edinburgh Castle, Edinburgh, Scotland  (55.9534, -3.1885)   
11               Bodleian Library, Oxford  (51.7575, -1.2447)   

    Travel Time to Kolkata (hours) Nearby Cricket Stadiums  
0                            12.68                   ## Se  
1                            12.67                   ## Se  
2                            12.67                   ## Se  
3                            12.68                   ## Se  
4                            12.70                   ## Se  
5                            12.67                   ## Se  
6                            12.69                   ## Se  
7                            12.88                   ## Se  
8                            12.78                   ## Se  
9                            12.86                   ## Se  
10                           12.79                   ## Se  
11                           12.78                   ## Se  

[Step 8: Duration 13.12 seconds| Input tokens: 41,664 | Output tokens: 2,905]

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


In [ ]:
result

,Location,Coordinates,Travel Time to Kolkata (hours),Nearby Cricket Stadiums
0,"King's Cross Station, London","(51.5307, -0.1257)",12.68,## Se
1,"Leadenhall Market, London","(51.5146, -0.0847)",12.67,## Se
2,"Millennium Bridge, London","(51.5063, -0.0901)",12.67,## Se
3,"Warner Bros. Studios, Leavesden","(51.7342, -0.2898)",12.68,## Se
4,"Hampton Court Palace, London","(51.4785, -0.3298)",12.70,## Se
5,"Alnwick Castle, Northumberland","(55.3705, -1.5938)",12.67,## Se
6,"Durham Cathedral, Durham","(54.7768, -1.5729)",12.69,## Se
7,"Gloucester Cathedral, Gloucestershire","(51.7414, -2.2413)",12.88,## Se
8,"Christ Church, Oxford","(51.7575, -1.2447)",12.78,## Se
9,"Lacock Abbey, Wiltshire","(51.1059, -1.7761)",12.86,## Se


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [ ]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to     │
│ here (we're in Kolkata, 22.5744° N, 88.3629° E), and return them to me as a pandas dataframe.                   │
│ Also give me some cricket stadiums if there are any nearby to the filming locations                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:393: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:402: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(
ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Our current location: Kolkata, 22.5744° N, 88.3629° E

### 1.2. Facts to look up
- List of all Harry Potter filming locations worldwide: This information can be obtained from reputable movie 
databases or dedicated Harry Potter fansites, such as "MuggleNet" or "Pottermore".
- Coordinates of each Harry Potter filming location: These must be found for each location listed in the previous 
fact.
- List of cricket stadiums near Harry Potter filming locations globally: This can be gathered from sports news 
websites, local tourism boards, or city-specific sports websites.

### 1.3. Facts to derive
- Travel time from each Harry Potter filming location to Kolkata via passenger plane: Using the coordinates from 
1.2 and a typical cruising speed for passenger aircraft, this value needs to be computed for each location.

---

## 2. Plan
- Perform a Web search for all Harry Potter filming locations worldwide.
- Visit each source URL from the web search results to verify and compile the accurate list of filming locations 
with geographic coordinates.
- Verify the geographic coordinates by cross-referencing them with official geographic services or dedicated 
mapping websites.
- Perform a Web search for cricket stadiums near each of the verified Harry Potter filming locations.
- Visit each relevant URL to confirm the existence and coordinates (if available) of the nearby cricket stadiums.
- Convert the verified coordinates for filming locations and nearby cricket stadiums into a pandas DataFrame 
format.
- Using the geographic coordinates and typical cruising speed for passenger planes, calculate the travel time to 
Kolkata for each filming location.
- Incorporate the calculated travel time into the DataFrame alongside other relevant details about the filming 
locations and nearby cricket stadiums.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


...

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: 
Root=1-68625897-433baa834521da4313493c89;6cb7dbc8-8dc2-485a-b327-5e2b9bdb46ca)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 9: Duration 0.21 seconds]

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-68625897-433baa834521da4313493c89;6cb7dbc8-8dc2-485a-b327-5e2b9bdb46ca)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [ ]:
detailed_report

,Location,Coordinates,Travel Time to Pune (hours),Nearby Cricket Stadiums
0,Durham Cathedral,None,None,## Search Results\n0. [Cricket Ground in Durha...
1,Alnwick Castle,None,None,## Search Results\n0. [ground details - Northu...
2,"Malham Cove, Skipton",None,None,## Search Results\n0. [Map of Yorkshire Cricke...
3,"New College, Oxford",None,None,## Search Results\n0. [New College Ground](htt...
4,Lacock Abbey,None,None,## Search Results\n0. [Cricket Grounds in Engl...
5,Virginia Water,None,None,## Search Results\n0. [Blackwater Cricket Asso...
6,"Freshwater West, Pembrokeshire, Wales",None,None,## Search Results\n0. [List of cricket grounds...
7,"Alnwick Castle, Northumberland, England",None,None,## Search Results\n0. [ground details - Northu...
8,Scottish Highlands,None,None,## Search Results\n0. [Cricket Grounds in Scot...
9,London,None,None,## Search Results\n0. [Cricket grounds in Lond...


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [ ]:
os.environ["SERPER_API_KEY"] = "" # Changed to SERPER_API_KEY and getting from userdata

model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o-mini", max_tokens=2000)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(

model=OpenAIServerModel("gpt-4o-mini", max_tokens=2000),
   tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
    "geopandas",
    "plotly",
    "plotly.io",  # explicitly add this
    "plotly.express",  # explicitly add this too
    "shapely",
    "json",
    "pandas",
    "numpy",
    "kaleido"
],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [ ]:
manager_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: ['geopandas', 'plotly', 'plotly.io', 'plotly.express', 'shapely', 'json', 'pandas', 
│   'numpy', 'kaleido']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │          

In [ ]:
!pip install markdownify requests
!pip install -U kaleido


manager_agent.run("""
ind all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here (we're in Kolkata, 22.5744° N, 88.3629° E), and return them to me as a pandas dataframe.
Also give me some cricket stadiums if there are any nearby to the filming locations. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ind all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here │
│ (we're in Kolkata, 22.5744° N, 88.3629° E), and return them to me as a pandas dataframe.                        │
│ Also give me some cricket stadiums if there are any nearby to the filming locations. You need at least 6 points │
│ in total.                                                                                                       │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Starting point for travel: Kolkata, coordinates (22.5744° N, 88.3629° E).
- Task requires a calculation of travel time via passenger plane to various Harry Potter filming locations.
- Task requests an inclusion of at least 6 points in total (Harry Potter filming locations and nearby cricket 
stadiums).
- Output format for the map: a pandas dataframe of the locations and a spatial map of the world.

### 1.2. Facts to look up
- List of Harry Potter filming locations around the world.
  - **Source:** Potentially findable on dedicated movie location websites, fandom sites for Harry Potter, or film 
tourism resources.
- Coordinates (latitude and longitude) of at least 6 Harry Potter filming locations.
- Locations of nearby cricket stadiums relative to the provided Harry Potter filming locations.
  - **Source:** Look up cricket stadium databases, cricket boards' websites, or sports tourism sites that list 
stadium locations.

### 1.3. Facts to derive
- Calculate the travel time for each Harry Potter filming location from Kolkata using the 
`calculate_cargo_travel_time` function.
- Gather and match cricket stadium coordinates to the identified Harry Potter filming locations to determine 
proxies or proximity.
- Create a pandas dataframe collating the obtained data of filming locations and their respective travel times.
- Prepare a spatial map visualization with a scatter plot of the filming locations.

## 2. Plan
1. Use the `web_agent` to find and collect a list of Harry Potter filming locations globally along with their 
coordinates.
2. Use the `web_agent` to find and collect cricket stadiums near the recorded Harry Potter filming locations, 
including their coordinates.
3. Combine the list of Harry Potter locations and cricket stadiums to ensure a minimum of 6 unique points.
4. For each identified filming location and cricket stadium, use the `calculate_cargo_travel_time` function to 
compute the travel time from Kolkata.
5. Store the filming locations, cricket stadiums, and their corresponding travel times in a pandas dataframe.
6. Utilize the gathered data to create a spatial map using Plotly Express to represent the filming locations and 
cricket stadiums.
7. Save the generated map as "saved_map.png".
8. Return the map visualization as the final answer through the `final_answer` function.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


...

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:393: FutureWarning:

Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:402: FutureWarning:

Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


Reached max steps.

[Step 16: Duration 25.79 seconds| Input tokens: 156,741 | Output tokens: 6,077]

ERROR:opentelemetry.exporter.otlp.proto.http.trace_exporter:Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


'To effectively address the user task, I will summarize the crucial steps involved in gathering Harry Potter filming locations, calculating travel times from Kolkata, identifying nearby cricket stadiums, and displaying this information through a spatial map. \n\nHere’s the detailed response:\n\n### Step 1: Identify Harry Potter Filming Locations\nHere are notable Harry Potter filming locations along with their geographical coordinates (latitude, longitude):\n\n1. **Alnwick Castle, England**: (55.4155, -1.6964)\n2. **Glenfinnan Viaduct, Scotland**: (56.3040, -4.1138)\n3. **King\'s Cross Station, London, England**: (51.5308, -0.1232)\n4. **Durham Cathedral, England**: (54.7754, -1.5764)\n5. **Christ Church, Oxford, England**: (51.7548, -1.2545)\n6. **Lacock Abbey, England**: (51.4234, -2.1312)\n\n### Step 2: Identify Nearby Cricket Stadiums\nCricket stadiums located near the filming sites include:\n\n1. **Lord\'s Cricket Ground, London**: (51.5254, -0.3884)\n2. **Durham County Cricket Cl

In [ ]:
manager_agent.python_executor.state["fig"]